In [11]:
from kaggle_environments import make, evaluate
import random

In [12]:
# Create the game environment
# Set debug=True to see the errors if your agent refuses to run
env = make('connectx', debug=True)

# List of available default agents
print(list(env.agents))

['random', 'negamax']


In [13]:
# Two random agents play one round of game
env.run(['random', 'random'])

# Render the game
env.render(mode='ipython')

In [14]:
import numpy as np


def get_opponent_mark(player_mark):
    return 2 if player_mark == 1 else 1


# Gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, piece, config):
    next_grid = grid.copy()
    for row in range(config.rows - 1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = piece
    return next_grid


def check_winning_move(obs, config, col, piece):
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    next_grid = drop_piece(grid, col, piece, config)
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(next_grid[row, col:col + config.inarow])
            if window.count(piece) == config.inarow:
                return True
    # vertical
    for row in range(config.rows - (config.inarow - 1)):
        for col in range(config.columns):
            window = list(next_grid[row:row + config.inarow, col])
            if window.count(piece) == config.inarow:
                return True
    # positive diagonal
    for row in range(config.rows - (config.inarow - 1)):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(next_grid[range(row, row + config.inarow), range(col, col + config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    # negative diagonal
    for row in range(config.inarow - 1, config.rows):
        for col in range(config.columns - (config.inarow - 1)):
            window = list(next_grid[range(row, row - config.inarow, -1), range(col, col + config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    return False

In [15]:
def agent_random(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return random.choice(valid_moves)


def agent_leftmost(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return valid_moves[0]


def agent_rightmost(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return valid_moves[-1]


opponent_moves = dict()


def defensive_agent(obs, config):
    my_piece = obs.mark
    enemy_piece = 1 if my_piece == 2 else 2
    for i, cell in enumerate(obs.board):
        if cell != my_piece and cell != 0:
            opponent_moves[i] = opponent_moves.get(i, 0) + cell

    opponent_last_move = None
    for cell, mark in opponent_moves.items():
        if mark == enemy_piece:
            opponent_last_move = cell

    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    if opponent_last_move is None:
        return random.choice(valid_moves)

    retaliation_ideas = []
    if opponent_last_move + 1 < len(obs.board):
        if obs.board[opponent_last_move + 1] == 0:
            if (opponent_last_move + 1) + 7 < len(obs.board):
                if obs.board[(opponent_last_move + 1) + 7] != 0:
                    retaliation_ideas.append(opponent_last_move + 1)
            else:
                retaliation_ideas.append(opponent_last_move + 1)

    if opponent_last_move - 1 >= 0:
        if obs.board[opponent_last_move - 1] == 0:
            if (opponent_last_move - 1) + 7 < len(obs.board):
                if obs.board[(opponent_last_move - 1) + 7] != 0:
                    retaliation_ideas.append(opponent_last_move - 1)
            else:
                retaliation_ideas.append(opponent_last_move - 1)

    if opponent_last_move % 7 in valid_moves:
        retaliation_ideas.append(opponent_last_move)

    retaliation_ideas = [col % 7 for col in retaliation_ideas]
    valid_ideas = [cell for cell in retaliation_ideas if cell in valid_moves]
    if len(valid_ideas) == 0:
        return random.choice(valid_moves)

    return random.choice(valid_ideas)


def cautious_agent(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]

    for move in valid_moves:
        if check_winning_move(obs, config, move, obs.mark):
            return move

    for move in valid_moves:
        if check_winning_move(obs, config, move, get_opponent_mark(obs.mark)):
            return move

    return random.choice(valid_moves)


In [21]:
env.run([cautious_agent, "negamax"])

env.render(mode="ipython")

In [17]:
def mean_reward(rewards):
    return sum(r[0] for r in rewards) / float(len(rewards))


print("Defensive Agent vs Random Agent: ",
      mean_reward(evaluate("connectx", [cautious_agent, "random"], num_episodes=100)))
print("Defensive Agent vs Negamax Agent: ",
      mean_reward(evaluate("connectx", [cautious_agent, "negamax"], num_episodes=10)))
print("Cautious Agent vs Defensive Agent: ",
      mean_reward(evaluate("connectx", [cautious_agent, defensive_agent], num_episodes=100)))

Defensive Agent vs Random Agent:  0.96
Defensive Agent vs Negamax Agent:  -0.3
Cautious Agent vs Defensive Agent:  0.96


In [46]:
# Calculates score if agent drops piece in selected column
def score_move(grid, col, mark, config):
    next_grid = drop_piece(grid, col, mark, config)
    score = get_heuristic(next_grid, mark, config)
    return score

# Helper function for score_move: gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid

# Helper function for score_move: calculates value of heuristic for grid
def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    num_fours_opp = count_windows(grid, 4, mark%2+1, config)
    num_twos_opp = count_windows(grid, 2, mark%2+1, config)
    score = num_threes - 3e2*num_threes_opp - (-1*num_twos_opp) + 1e6*num_fours
    #score = num_threes_opp - 1e3*num_threes - 1e6*num_fours + 1e6*num_fours_opp
    return score

# Helper function for get_heuristic: checks if window satisfies heuristic conditions
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

In [47]:
def simulate_dropping_piece(grid, move, rows, mark):
    next_grid = grid.copy()
    for row in range(rows-1, -1, -1):
        if next_grid[row, move] == 0:
            next_grid[row, move] = mark
            break
            
    return next_grid

def heuristics_agent(obs, config):
    valid_moves = [move for move in range(config.columns) if obs.board[move] == 0]
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    
    move_scores = []
    for move in valid_moves:
        move_scores.append(get_heuristic(simulate_dropping_piece(grid, move, config.rows, obs.mark), obs.mark, config))
    scores = dict(zip(valid_moves, move_scores))
    
    max_scores = [key for key in scores.keys() if scores[key] == max(scores.values())]
    
    return random.choice(max_scores)

env.run([heuristics_agent, heuristics_agent])

env.render(mode="ipython")

In [20]:
import numpy as np

flat_grid = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0]
flat_grid = [x for x in range(42)]
# print(len(flat_grid))
grid = np.asarray(flat_grid).reshape(6, 7)
print(grid)
# print([x for x in range(3, 6)])
# print([x for x in range(0, 4)])
# print(list(grid[range(0, -4, -1), range(0, 4)]))

inarow = 4
rows = 6
columns = 7

# negative diagonal
for row in range(inarow-1, rows):
    for col in range(columns-(inarow-1)):
        window = list(grid[range(row, row-inarow, -1), range(col, col+inarow)])
        print(window)
    #     window = list(grid[range(row), range(col+inarow)])
    #     print(window)

# horizontal
# for row in range(6):
#     for col in range(7-(4-1)):
#         window = list(grid[0, 0:0+4])
#         if check_window(window, num_discs, piece, config):
#             num_windows += 1


# positive diagonal
# for row in range(3):
#     for col in range(4):
#         window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
#         if check_window(window, num_discs, piece, config):
#             num_windows += 1

[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]
 [35 36 37 38 39 40 41]]
[21, 15, 9, 3]
[22, 16, 10, 4]
[23, 17, 11, 5]
[24, 18, 12, 6]
[28, 22, 16, 10]
[29, 23, 17, 11]
[30, 24, 18, 12]
[31, 25, 19, 13]
[35, 29, 23, 17]
[36, 30, 24, 18]
[37, 31, 25, 19]
[38, 32, 26, 20]
